<mark>Antes de revisar este cuaderno sugiero chequear `GuardarANN.ipynb`.</mark>

# Elaboración del modelo

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pandas as pd #permite manipular los dataframes, que serían el equivalente a los Excel
import numpy as np #en caso se necesiten cálculos numéricos, como p. ej. encontrar valores nulos 
from sklearn.model_selection import train_test_split #para dividir datos en entrenamiento y testeo
from sklearn import preprocessing #para escalar/normalizar variables numéricas
import tensorflow as tf #Esta y las de abajo las necesitamos siempre para ANN y RF
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #para calcular parámetros del modelo
import pickle
import numpy as np

Como ya había guardado los objetos que contienen mis $X$ y mi $y$, sólo los cargo.

In [2]:
Xtrain_scale=np.load('data/Xtrain_scale.npy')
ytrain_array=np.load('data/ytrain_array.npy')

Xtest_scale=np.load('data/Xtest_scale.npy')
ytest_array=np.load('data/ytest_array.npy')

Los concateno porque en un random forest, al usar muestras out-of-bag (OOB), ya no necesito muestras de entrenamiento y prueba por separado.

In [3]:
X_RF = np.concatenate([Xtrain_scale, Xtest_scale], axis=0)
y_RF = np.concatenate([ytrain_array, ytest_array], axis=0)

Creo el modelo.

In [4]:
# Crear el modelo de Random Forest
rf_model = RandomForestRegressor(criterion="squared_error",
                                 n_estimators=50, #N de árboles. Literatura indica que esto es prueba y error 
                                 max_depth=None,#Se expandirá en función del siguiente parámetro
                                 min_samples_split=2, 
                                 min_samples_leaf=1, 
                                 max_features=None, #Cantidad máxima de columnas
                                 max_samples=None, #Cantidad máxima de observaciones
                                 bootstrap=True,
                                 oob_score=True, #Muestras OOB
                                 random_state=42, #Semilla de aleatorización. Puede ser cualquier número
                                 )
# Entrenar el modelo
rf_model.fit(X_RF, y_RF)

RandomForestRegressor(max_features=None, n_estimators=50, oob_score=True,
                      random_state=42)

Reviso las métricas.

In [5]:
#Calculo la cantidad de datos que tienen las muestras OOB
num_oob_samples = len(y_RF) - int(len(y_RF) * rf_model.oob_score_)

# Obtener las predicciones del modelo para las muestras OOB
oob_predictions = rf_model.oob_prediction_

# Calcular el coeficiente de determinación (R2)
r2RF = r2_score(y_RF, oob_predictions)

# Calcular n y k
n = num_oob_samples  # Número de observaciones
k = X_RF.shape[1]  # Número de variables independientes

# Calcular R^2 ajustado
r2RF_adjusted = 1 - ((1 - r2RF) * (n - 1) / (n - k - 1))

# Calcular el error cuadrático medio (RMSE)
rmseRF = np.sqrt(mean_squared_error(y_RF, oob_predictions))

# Imprimir resultados
print("Coefficient of Determination (R2):", r2RF)
print("Adjusted Coefficient of Determination (R2 adj):", r2RF_adjusted)
print("Root Mean Squared Error (RMSE):", rmseRF)

Coefficient of Determination (R2): 0.8664242929641404
Adjusted Coefficient of Determination (R2 adj): 0.8056737595382268
Root Mean Squared Error (RMSE): 0.2032322313948567


Guardo en formato *.pkl*.

In [6]:
import joblib

# Guardar el modelo de Random Forest
joblib.dump(rf_model, 'model/rf.pkl')

['model/rf.pkl']

# Cargo el modelo

Nos pondremos en la misma situación que con el cuaderno anterior: he obtenido el modelo pero cerré mi sesión en Jupyter Lab y quiero revisar sus métricas de manera idéntica a lo que obtuve antes. Para ello importo el modelo en el objeto `loaded_rf_model`.

In [7]:
# Cargar el modelo de Random Forest
loaded_rf_model = joblib.load('model/rf.pkl')

Volveré a cargar mis arrays y concatenarlos.

In [8]:
Xtrain_scale=np.load('data/Xtrain_scale.npy')
ytrain_array=np.load('data/ytrain_array.npy')

Xtest_scale=np.load('data/Xtest_scale.npy')
ytest_array=np.load('data/ytest_array.npy')

X_RF = np.concatenate([Xtrain_scale, Xtest_scale], axis=0)
y_RF = np.concatenate([ytrain_array, ytest_array], axis=0)

Y ahora calculo las métricas sobre `loaded_rf_model`.

In [9]:
#Calculo la cantidad de datos que tienen las muestras OOB
num_oob_samples = len(y_RF) - int(len(y_RF) * loaded_rf_model.oob_score_) #noten que llamo a 'loaded_rf_model'

# Obtener las predicciones del modelo para las muestras OOB
oob_predictions = loaded_rf_model.oob_prediction_ #noten que llamo a 'loaded_rf_model'

# Calcular el coeficiente de determinación (R2)
r2RF = r2_score(y_RF, oob_predictions)

# Calcular n y k
n = num_oob_samples  # Número de observaciones
k = X_RF.shape[1]  # Número de variables independientes

# Calcular R^2 ajustado
r2RF_adjusted = 1 - ((1 - r2RF) * (n - 1) / (n - k - 1))

# Calcular el error cuadrático medio (RMSE)
rmseRF = np.sqrt(mean_squared_error(y_RF, oob_predictions))

# Imprimir resultados
print("Coefficient of Determination (R2):", r2RF)
print("Adjusted Coefficient of Determination (R2 adj):", r2RF_adjusted)
print("Root Mean Squared Error (RMSE):", rmseRF)

Coefficient of Determination (R2): 0.8664242929641404
Adjusted Coefficient of Determination (R2 adj): 0.8056737595382268
Root Mean Squared Error (RMSE): 0.2032322313948567
